### Construct descriptors from structure data

In [26]:
# Imports
import os
import json
import numpy as np
from ase.io import read
from ase import Atoms
from preprocess import load_json
from dscribe.descriptors import LMBTR

ROOT_DIR = os.getcwd()

In [18]:
# Load the structure data for Catalysis-hub data, and
# convert the structures into lists of ase.Atoms objects,
# where each list of Atoms objects represents one reaction.
# The reactions and the corresponding structures are in
# the same order.

with open(f"{ROOT_DIR}/data/reactions_cathub.json", "r") as file:
    STRUCT_DATA = json.load(file)
    
KEYS = list(STRUCT_DATA.keys())
STRUCTURES = []
for key in KEYS:
    structs = STRUCT_DATA[key]["structures"]
    reaction_structures = []
    for struct in structs:
        struct_dict = {}
        with open(f"{ROOT_DIR}/data/struct_tmp.xyz", "w") as f:
            f.write(struct["InputFile"])
        atoms = read(f"{ROOT_DIR}/data/struct_tmp.xyz")
        struct_dict["atoms"] = atoms
        struct_dict["energy"] = struct["energy"]
        reaction_structures.append(struct_dict)
    STRUCTURES.append(reaction_structures)
os.remove(f"{ROOT_DIR}/data/struct_tmp.xyz")

print(f"Loaded {len(STRUCTURES)} corresponding structure lists.")

Loaded 1431 corresponding structure lists.


In [21]:
def get_init_fin_structures(structure):
    """
    The function finds the initial and final structures from a set of
    structures describing the given reaction. Only structures comprising
    more than 10 atoms are used to discard any structures that do not
    describe the whole system. From these, the first and the last
    structure are chosen as the initial and final structures, respectively.
    
    Note: This method might be incorrect, as some structures might have
    10 or more atoms, but still describe an incomplete adsorbed system.
    Also, the order of the structures is not confirmed, so the first and
    the last structures might not correctly represent the initial and
    final configurations of the system.
    
    Params:
      structure (list):  A list of dictionaries describing the individual
                          structures of the reaction.
    Returns:
      ret (list):        A list containing the dictionaries for the first
                          and the last structure in the reaction.
      None:              If there are less than two structures available.
    """
    
    struct_list = []
    # Select only structures that have at least 10 atoms
    for i,struct in enumerate(structure):
        if len(struct["atoms"]) >= 10:
            struct_list.append(struct)
    # Return the first and the last structure only if more than two
    # structures are found
    if len(struct_list) > 1:
        ret = [struct_list[0], struct_list[-1]]
    else:
        ret = None
        
    return ret

In [20]:
# Get the initial and final configurations from reactions
INIT_FIN_STRUCTURES = []
INDICES = []
for i,struct in enumerate(STRUCTURES):
    res = get_init_fin_structures(struct)
    if res:
        INIT_FIN_STRUCTURES.append(res)
        INDICES.append(i)

INIT_FIN_STRUCTURES = np.array(INIT_FIN_STRUCTURES)
DF_CATHUB_RAW = load_json(f"{ROOT_DIR}/data/reactions_cathub.json")
INIT_FIN_DATA = DF_CATHUB_RAW.iloc[INDICES]

print(f"Found {len(INIT_FIN_STRUCTURES)} reactions with initial and final configurations.")

Loaded 1431 reactions from file /home/hlappal/projects/catalysis_data_project/data/reactions_cathub.json
Found 506 reactions with initial and final configurations.


In [22]:
def build_struct_descriptor(structure):
    """
    The function builds a combined LMBTR-based descriptor for a given reaction.
    The descriptor consists of two individual LMBTR descriptors that are
    concatenated into a single 1D array.
    
    Params:
      structure (list):     The dictionaries for the first and the last configuration.
    Returns:
      total_lmbtr (array):  Numpy array of the combined LMBTR descriptors.
    """
    
    init_struct = Atoms(structure[0]["atoms"])
    fin_struct  = Atoms(structure[1]["atoms"])
    species = []
    species.extend(init_struct.get_chemical_symbols())
    species.extend(fin_struct.get_chemical_symbols())
    species = list(dict.fromkeys(species))
    periodic = init_struct.get_pbc().any()
    lmbtr = LMBTR(
        species=species,
        k2={
            "geometry": {"function": "distance"},
            "grid": {"min": 0, "max": 5, "n": 50, "sigma": 0.005},
            "weighting": {"function": "exponential", "scale": 0.5, "cutoff": 1e-3},
        },
        k3={
            "geometry": {"function": "angle"},
            "grid": {"min": 0, "max": 180, "n": 50, "sigma": 0.005},
            "weighting": {"function": "exponential", "scale": 0.5, "cutoff": 1e-3},
        },
        periodic=periodic,
        normalization="l2_each",
    )
    init_lmbtr = lmbtr.create(init_struct).flatten()
    fin_lmbtr = lmbtr.create(fin_struct).flatten()
    total_lmbtr = np.concatenate((init_lmbtr, fin_lmbtr), axis=None)
    
    return total_lmbtr

In [27]:
# Build a list of structural descriptors and the corresponding activation energies
DESCRIPTORS = []
TARGETS = []
for i in range(len(INIT_FIN_STRUCTURES)):
    descr = build_struct_descriptor(INIT_FIN_STRUCTURES[i])
    DESCRIPTORS.append(descr)
    TARGETS.append(INIT_FIN_DATA.iloc[i]["activationEnergy"])
print(f"Built {len(DESCRIPTORS)} descriptors with corresponding targets")

Built 506 descriptors with corresponding targets


In [ ]:
# Pad ragged lists with zeros
MAX_LEN = 0
for row in DESCRIPTORS:
    if len(row) > MAX_LEN:
        MAX_LEN = len(row)
print(f"Number of features: {MAX_LEN}")
for i,row in enumerate(DESCRIPTORS):
    DESCRIPTORS[i] = list(row) + [0 for i in range(MAX_LEN - len(row))]
DESCRIPTORS = np.array(DESCRIPTORS)

Number of features: 296000


In [ ]:
# Save the LMBTR structure descriptors and targets
np.savetxt(f"{ROOT_DIR}/data/structure_descriptors.txt", DESCRIPTORS)
np.savetxt(f"{ROOT_DIR}/data/structure_targets.txt", TARGETS)